In [13]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml
import git

from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml
import git

import cube_parser.cube as cube

from droughty.lookml_base_dict import d1

def get_all_values(nested_dictionary):

    
    for key,value in nested_dictionary.items():

        explore = {


            "cube": key,
            "sql": "select * from"+" "+key,
            
            "measures": "{"
                
        }
            
        
        yield(cube.dump(explore))
        

        for key, value in value.items():
            
            if "pk" not in key[0] and "fk" not in key[0] and "date" not in key[1] and "timestamp" not in key[1] and "number" not in key [1]:


                dimension = {


                    "dimension": {
                    "sql": key[0],
                    "type": key[1],
                    "name": key[0]

                    }

                }

                yield(cube.dump(dimension))

                    
for value in get_all_values(nested_dictionary):

    print(value)

cube (`budgets_sales `, {
sql: `select * from budgets_sales`,
measures: {
budget_group:  {
  sql: `budget_group`,
  type: `string`,
} ,
budget_name:  {
  sql: `budget_name`,
  type: `string`,
} ,
intake_name:  {
  sql: `intake_name`,
  type: `string`,
} ,
institution_name:  {
  sql: `institution_name`,
  type: `string`,
} ,
entry_type_code:  {
  sql: `entry_type_code`,
  type: `string`,
} ,
sales_region_group:  {
  sql: `sales_region_group`,
  type: `string`,
} ,
cube (`core_agents `, {
sql: `select * from core_agents`,
measures: {
agent_salesforce_business_key:  {
  sql: `agent_salesforce_business_key`,
  type: `string`,
} ,
name:  {
  sql: `name`,
  type: `string`,
} ,
group:  {
  sql: `group`,
  type: `string`,
} ,
status:  {
  sql: `status`,
  type: `string`,
} ,
sales_channel:  {
  sql: `sales_channel`,
  type: `string`,
} ,
uk_based_agent:  {
  sql: `uk_based_agent`,
  type: `yesno`,
} ,
cube (`core_countries `, {
sql: `select * from core_countries`,
measures: {
country:  {
  sql